In [56]:
# Import Libraries and define functions
import csv
import numpy as np
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [15, 5]
plt.rcParams.update({'font.size': 22})

def set_style():
    plt.minorticks_on()
    plt.grid(visible=True, which='minor', color='white', linestyle='-', alpha=0.4)
    plt.style.use('ggplot')
    plt.rcParams.update({'font.size': 22})
    legend = plt.gca().legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=2, mode="expand", borderaxespad=0., frameon = 1)
    frame = legend.get_frame()
    frame.set_facecolor('white')
    frame.set_edgecolor('white')

def save_fig(file_name):
    plt.savefig(file_name, dpi=300, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches='tight', pad_inches=0.02,
        frameon=None, metadata=None)

def read_file(path):
  with open(path, 'r') as fd:
    cwriter = csv.reader(fd)
    data = []
    skipped_first_row = False
    for row in cwriter:
      if not skipped_first_row:
        skipped_first_row = True
        continue
      row_tmp = row
      for idx in range(len(row_tmp)):
        row_tmp[idx] = float(row_tmp[idx])
      data.append(row_tmp)
    #print(data)
    config = data[0]
    data_run = data[1]
    data_energy = data[2]
    #print(config)
    #print(data_run)
    #print(data_energy)
  return data_run, data_energy, config

# def read_file(path):
#     with open(path, 'r') as fd:
#         cwriter = csv.reader(fd, delimiter=',')  # Use space as the delimiter
#         data = []
#         skipped_first_row = False
#         for row in cwriter:
#             if not skipped_first_row:
#                 skipped_first_row = True
#                 continue
#             row_tmp = [float(val.strip()) for val in row if val.strip()]  # Remove empty values
#             print(row_tmp)
#             data.append(row_tmp)
#         config = data[0]
#         data = data[1]
#     return data, config

def read_data_folder(path):
  data_run_max = []
  data_energy_max = []
  data_run_all = []
  data_energy_all = []
  configs = []
  for idx in range(1, DESIGNS+1):
    data_run, data_energy, config = read_file(path + '{}.csv'.format(idx))
    #go through running speed rewards
    for d in data_run:
        data_run_all.append(d)
    data_run_max.append(np.amax(data_run))
    data_run = np.array(data_run)
    data_run_all.append(data_run)
    #go through energy consumption rewards
    for d in data_energy:
        data_energy_all.append(d)
    data_energy_max.append(np.amax(data_energy))
    data_energy = np.array(data_energy)
    data_energy_all.append(data_energy)
    
    configs.append(config)
  return data_run_all, data_energy_all, data_run_max, data_energy_max, configs
    
def compute_mean_std(paths):
  data_max_ar_run = []
  #data_top_mean_ar_run = []
  #configs_ar_run = []
  data_max_ar_energy = []
  #data_top_mean_ar_energy = []
  configs_ar = []
  data_all_ar_run = []
  data_all_ar_energy = []
  for p in paths:
    data_run_all, data_energy_all, data_run_max, data_energy_max, configs = read_data_folder(p)
    #running speed
    data_all_ar_run.append(data_run_all)
    data_max_ar_run.append(data_run_max)
    #energy consumption
    data_all_ar_energy.append(data_energy_all)
    data_max_ar_energy.append(data_energy_max)
    configs_ar.append(configs)
    
  data_max_mean_run = np.mean(data_max_ar_run, axis=0)
  data_max_std_run = np.std(data_max_ar_run, axis=0)
  data_max_mean_energy = np.mean(data_max_ar_energy, axis=0)
  data_max_std_run_energy = np.std(data_max_ar_energy, axis=0)
  
  return data_max_mean_run, data_max_std_run, data_max_mean_energy, data_max_std_run_energy

def plot_data_means(mean_data, std_data, color, label='', plot_random_std=False):
  x = np.arange(1, len(mean_data) + 1)
  init_data_mean = mean_data[0:INITIAL_DESIGNS+1]
  init_data_std = std_data[0:INITIAL_DESIGNS+1]
  init_x = x[0:INITIAL_DESIGNS+1]
  optim_data_mean = mean_data[INITIAL_DESIGNS::2]
  optim_data_std = std_data[INITIAL_DESIGNS::2]
  optim_x = x[INITIAL_DESIGNS::2]
  random_data_mean = mean_data[INITIAL_DESIGNS+1::2]
  random_data_std = std_data[INITIAL_DESIGNS+1::2]
  random_x = x[INITIAL_DESIGNS+1::2]
  plt.plot(init_x, init_data_mean, color=color, alpha=0.3, linewidth=2.0)
  plt.fill_between(init_x, init_data_mean - init_data_std, init_data_mean + init_data_std, facecolor=color, alpha=0.1)
  plt.plot(optim_x, optim_data_mean, color=color, label=label, linewidth=2.0)
  plt.fill_between(optim_x, optim_data_mean - optim_data_std, optim_data_mean + optim_data_std, facecolor=color, alpha=0.2)
  #plt.plot(random_x, random_data_mean, color=color, linestyle='--')
  if plot_random_std:
    plt.fill_between(random_x, random_data_mean - random_data_std, random_data_mean + random_data_std, facecolor=color, alpha=0.2)
  set_style()

def plot_data_means_optim_random(mean_data, std_data, color, label=''):
  x = np.arange(1, len(mean_data) + 1)
  init_data_mean = mean_data[0:INITIAL_DESIGNS+1]
  init_data_std = std_data[0:INITIAL_DESIGNS+1]
  init_x = x[0:INITIAL_DESIGNS+1]
  optim_data_mean = mean_data[INITIAL_DESIGNS::2]
  optim_data_std = std_data[INITIAL_DESIGNS::2]
  optim_x = x[INITIAL_DESIGNS::2]
  random_data_mean = mean_data[INITIAL_DESIGNS+1::2]
  random_data_std = std_data[INITIAL_DESIGNS+1::2]
  random_x = x[INITIAL_DESIGNS+1::2]
  plt.plot(init_x, init_data_mean, color=color, alpha=0.3, linewidth=2.0)
  plt.fill_between(init_x, init_data_mean - init_data_std, init_data_mean + init_data_std, facecolor=color, alpha=0.1)
  plt.plot(optim_x, optim_data_mean, color=color, linewidth=2.0, label=label)
  plt.fill_between(optim_x, optim_data_mean - optim_data_std, optim_data_mean + optim_data_std, facecolor=color, alpha=0.2)
  plt.plot(random_x, random_data_mean, color=color, label=label + ' Random Exploration', linestyle='--', linewidth=2.0)
  plt.fill_between(random_x, random_data_mean - random_data_std, random_data_mean + random_data_std, facecolor=color, alpha=0.2)
  set_style()

In [57]:
# Change these parameters as required
INITIAL_DESIGNS = 3 # 5
DESIGNS = 60 #50 

# Proposed method using data-driven design optimization
# Change folder names
EXPERIMENT_FOLDERS_1 = [
    #'data_exp_sac_pso_batch/Mon_Oct_21_05:47:55_2019__e37381f2',
    #'data_exp_sac_pso_batch/Mon_Oct_21_05:48:30_2019__0041f8c5',
    #'data_exp_sac_pso_batch/Mon_Oct_21_05:49:00_2019__5e2e3460',
    #'data_exp_sac_pso_batch/Sun_Oct_20_23:36:49_2019__5db36c30',
    #'data_exp_sac_pso_batch/Sun_Oct_20_23:37:06_2019__30302df7',
    '/home/oskar/Thesis/Results_scalarized/results_with_wandb/Wed_Oct_18_18:29:24_2023__a18577fe_1_0',
    #'/home/oskar/Thesis/Results_scalarized/results_with_wandb/Wed_Oct_18_18:29:14_2023__49c4bc4c_.9_.1',
    #'/home/oskar/Thesis/Results_scalarized/results_with_wandb/Fri_Oct_20_17:52:03_2023__d44355ca_.8_.2',
    #'/home/oskar/Thesis/Results_scalarized/results_with_wandb/Fri_Oct_20_17:52:03_2023__2c381a80_.7_.3',
    #'/home/oskar/Thesis/Results_scalarized/results_with_wandb/Fri_Oct_20_17:52:03_2023__d79ce48d_.6_.4',
    #'/home/oskar/Thesis/Results_scalarized/results_with_wandb/Fri_Oct_20_17:52:03_2023__fb9514fe_.5_.5',
    
]

# Baseline using simulations for the evaluation of design candidates
# Change folder names
EXPERIMENT_FOLDERS_2 = [
    '/home/oskar/Thesis/Results_scalarized/results_with_wandb/Wed_Oct_18_18:29:14_2023__49c4bc4c_.9_.1',
    #'data_exp_sac_pso_sim/Thu_Sep_28_09:27:21_2023__18d61b3d',
    #'data_exp_sac_pso_sim/Wed_Oct_23_05:49:22_2019__6878474d',
    #'data_exp_sac_pso_sim/Wed_Oct_23_05:51:21_2019__36c0fafc',
    #'data_exp_sac_pso_sim/Wed_Oct_23_05:51:33_2019__b21f861b',
    #'data_exp_sac_pso_sim/Wed_Oct_23_05:51:45_2019__cc74a513',
]


In [58]:
%matplotlib

Using matplotlib backend: TkAgg


In [59]:
exp_files = ['{}/data_design_'.format(folder) for folder in EXPERIMENT_FOLDERS_1] # Novelty Search + PSO on Q
exp_run_mean, exp_run_std, exp_energy_mean, exp_energy_std  = compute_mean_std(exp_files)
#print(exp_run_mean)
#print(exp_run_std)
#print()
#print(exp_energy_mean)
#print(exp_envergy_std)

In [60]:
# Plot performance of optimized designs and randomly selected designs
#plt.figure()
plt.figure(figsize=(15, 7))
plt.subplot(1, 2, 1)
plot_data_means_optim_random(exp_run_mean, exp_run_std, color='red', label='Proposed Method')
plt.ylabel('Cum. Episodic Reward for running speed')
plt.xlabel('Designs')
plt.ylim([50,700])
plt.xlim([1,DESIGNS])
plt.legend(loc='lower left')
#plt.show()
#plt.figure()

plt.subplot(1, 2, 2)
plot_data_means_optim_random(exp_energy_mean, exp_energy_std, color='red', label='Proposed Method')
plt.ylabel('Cum. Episodic Reward for energy consumption')
plt.xlabel('Designs')
plt.ylim([50,4500])
plt.xlim([1,DESIGNS])
plt.legend(loc='lower left')
#plt.show()
#plt.tight_layout()

#plt.subplots_adjust(wspace=0.4)
plt.show()
#save_fig('plots_HalfCheetah_random_vs_novelty_search.pdf')

In [61]:
# Compare two experiments/methods against each other
plt.figure()
exp_files = ['{}/data_design_'.format(folder) for folder in EXPERIMENT_FOLDERS_1] # Novelty Search + PSO on Q
exp_run_mean, exp_run_std, exp_energy_mean, exp_energy_std = compute_mean_std(exp_files)
plot_data_means(exp_run_mean, exp_run_std, color='red', label='Proposed Method')

exp_files = ['{}/data_design_'.format(folder) for folder in EXPERIMENT_FOLDERS_2] # Novelty Search + PSO on Q
exp_run_mean, exp_run_std, exp_energy_mean, exp_energy_std = compute_mean_std(exp_files)
plot_data_means(exp_run_mean, exp_run_std, color='blue', label='Using Simulations')

plt.ylabel('Cum. Episodic Reward Running Speed')
plt.xlabel('Designs')
plt.ylim([150,700])
plt.xlim([1,DESIGNS])
plt.show()
plt.figure()

exp_files = ['{}/data_design_'.format(folder) for folder in EXPERIMENT_FOLDERS_1] # Novelty Search + PSO on Q
exp_run_mean, exp_run_std, exp_energy_mean, exp_energy_std = compute_mean_std(exp_files)
plot_data_means(exp_energy_mean, exp_energy_std, color='red', label='Proposed Method')

exp_files = ['{}/data_design_'.format(folder) for folder in EXPERIMENT_FOLDERS_2] # Novelty Search + PSO on Q
exp_run_mean, exp_run_std, exp_energy_mean, exp_energy_std = compute_mean_std(exp_files)
plot_data_means(exp_energy_mean, exp_energy_std, color='blue', label='Using Simulations')

plt.ylabel('Cum. Episodic Reward Energy Consumption')
plt.xlabel('Designs')
plt.ylim([150,4500])
plt.xlim([1,DESIGNS])
plt.show()